In [ ]:
from pathlib import Path

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tardis.io.atom_data import AtomData
from tardis.io.configuration.config_reader import Configuration

home = Path.home()

ION_SLICE = (1, slice(None), slice(None), slice(None))

# identical atomic data to that used by C Vogl
atom_data = AtomData.from_hdf(
    "/storage/shield90/merged_mod_20SNG_forbidden_yg_fix_H30_cmfgen_yg_CONVERTED.h5"
)  # currently not available for public use

atom_data.prepare_atom_data([1], "macroatom", [(1, 0)], [(1, 0)])

config = Configuration.from_yaml(
    home / "tardis/tardis/plasma/tests/data/plasma_base_test_config.yml"
)

config.supernova.time_explosion = 16.084 * u.day
config.model.structure.type = "file"
config.model.structure.filename = (
    home
    / "tardis/docs/physics/plasma/equilibrium/cmfgen_stephane_density_rebin.dat"
)
config.model.structure.filetype = "simple_ascii"
config.model.structure.v_inner_boundary = 10000 * u.km / u.s
config.model.structure.v_outer_boundary = 15000 * u.km / u.s

config.model.abundances.He = 0
config.model.abundances.H = 1

config.plasma.excitation = "dilute-lte"
config.plasma.ionization = "nebular"

config.plasma.continuum_interaction.species = ["H 1"]
config.plasma.nlte.species = [(1, 0)]
config.plasma.nlte_ionization_species = ["H 1"]
config.plasma.nlte_excitation_species = ["H 1"]

In [ ]:
time_simulation = 7.2671371e-44 * u.s
volume = 1.61751052e44 * np.ones(1) * u.cm**3

### Set up electrons and radiation field

First plasma solution BEFORE MC step

In [ ]:
from tardis.plasma.electron_energy_distribution import (
    ThermalElectronEnergyDistribution,
)
from tardis.plasma.radiation_field import (
    DilutePlanckianRadiationField,
)

radiation_temp = 9984.961312868334 * np.ones(1) * u.K
dilution_factor = 0.1863524378417558 * np.ones(1)

electron_temp = 9984.961312868334 * np.ones(1)
electron_density = 2206775091.3630457 * np.ones(1)

elemental_number_density = pd.DataFrame(2206918615.4642744 * np.ones(1), index=[1])
elemental_number_density.index.name = "atomic_number"

thermal_electron_distribution = ThermalElectronEnergyDistribution(
    0 * u.erg, electron_temp * u.K, electron_density * u.cm**-3
)
radiation_field = DilutePlanckianRadiationField(radiation_temp, dilution_factor)

# Set up atomic data for compatibility

In [ ]:
from tardis.iip_plasma.continuum.base_continuum_data import ContinuumData

atom_data.continuum_data = ContinuumData(
               atom_data, selected_continuum_species=[(1, 0)]
           )

atom_data.Yg_data = atom_data.yg_data

atom_data.ionization_data.index = atom_data.ionization_data.index.set_levels(
    atom_data.ionization_data.index.levels[0].astype(int), level=0
).set_levels(
    atom_data.ionization_data.index.levels[1].astype(int), level=1
)

atom_data.lines.index = atom_data.lines.index.set_levels(
    atom_data.lines.index.levels[0].astype(int), level=0
).set_levels(
    atom_data.lines.index.levels[1].astype(int), level=1
).set_levels(
    atom_data.lines.index.levels[2].astype(int), level=2
).set_levels(
    atom_data.lines.index.levels[3].astype(int), level=3
)

atom_data.nlte_data._init_indices()

atom_data.has_collision_data = False

### Set up plasma

In [ ]:
from tardis.iip_plasma.standard_plasmas import LegacyPlasmaArray

plasma = LegacyPlasmaArray(
    elemental_number_density,
    atom_data,
    config.supernova.time_explosion.to("s").value,
    nlte_config=config.plasma.nlte,
    delta_treatment=None,
    ionization_mode="nlte",
    excitation_mode="dilute-lte",
    line_interaction_type=config.plasma.line_interaction_type,
    link_t_rad_t_electron=1.0,
    # link_t_rad_t_electron=self.ws**0.25,
    helium_treatment="none",
    heating_rate_data_file=None,
    v_inner=None,
    v_outer=None,
    continuum_treatment=True,
)

In [ ]:
j_blues_ctardis = pd.read_csv(
    "/home/afullard/tardis-chvogl-configs/j_blues.csv", index_col=0
)


In [ ]:
plasma.update_radiationfield(
            radiation_temp, dilution_factor, j_blues_ctardis["0"],
            config.plasma.nlte, initialize_nlte=True,
            n_e_convergence_threshold=0.05, **{})

In [ ]:
plasma.ion_number_density

In [ ]:
plasma.level_number_density.head()

In [ ]:
plasma.partition_function

In [ ]:
plasma.general_level_boltzmann_factor.head()

In [ ]:
plasma.electron_densities

In [ ]:
plasma.phi

In [ ]:
plasma.level_boltzmann_factor.head()

# Update plasma following ctardis

Also update radiation field and electron distribution

In [ ]:
radiation_field.temperature = 9992.27229695 * np.ones(1) * u.K
radiation_field.dilution_factor = 0.3571996 * np.ones(1)

thermal_electron_distribution.number_density = (
    2206775091.3630457 * np.ones(1) * u.cm**-3
)
thermal_electron_distribution.temperature = 9992.27229695 * np.ones(1) * u.K

# Load $J_{blues}$ and create an estimated radiation field

# Full NLTE and continuum calculation of the level Boltzmann factor

In [ ]:
photo_ion_estimator = pd.read_csv(
    "/home/afullard/tardis-chvogl-configs/photo_ion_estimator.csv",
    index_col=(0),
)
stim_recomb_estimator = pd.read_csv(
    "/home/afullard/tardis-chvogl-configs/stim_recomb_estimator.csv",
    index_col=(0),
)

photo_ion_estimator.columns = photo_ion_estimator.columns.astype(int)
stim_recomb_estimator.columns = stim_recomb_estimator.columns.astype(int)
# Create MultiIndex for photo_ion_estimator
photo_ion_estimator_idx = pd.MultiIndex.from_tuples(
    [(1, 0, level) for level in photo_ion_estimator.index],
    names=["atomic_number", "ion_number", "level_number"],
)
photo_ion_estimator.index = photo_ion_estimator_idx

# Create MultiIndex for stim_recomb_estimator
stim_recomb_estimator_idx = pd.MultiIndex.from_tuples(
    [(1, 0, level) for level in stim_recomb_estimator.index],
    names=["atomic_number", "ion_number", "level_number"],
)
stim_recomb_estimator.index = stim_recomb_estimator_idx

In [ ]:
data_path = home / "tardis-regression-data/testdata/thermal_data"
bf_heating_estimator = pd.read_csv(
    data_path / "thermal_bf_heating_est.csv", index_col=(0, 1, 2)
)
stim_recomb_cooling_estimator = pd.read_csv(
    data_path / "thermal_stim_cooling_est.csv", index_col=(0, 1, 2)
)

stim_recomb_cooling_estimator = pd.read_csv(
    data_path / "thermal_stim_cooling_est.csv", index_col=(0, 1, 2)
)

coll_deexc_heating_estimator = pd.read_csv(
    data_path / "coll_deexc_heating_estimator.csv", index_col=(0)
)


ff_heating_estimator = [
    4.89135279e-24,
    4.37696370e-24,
    3.75869301e-24,
    4.97847160e-24,
    4.52158002e-24,
    4.21024499e-24,
    3.94991540e-24,
    3.72915649e-24,
    3.58902110e-24,
    3.40170224e-24,
    3.20848519e-24,
    3.03540032e-24,
    2.87314722e-24,
    2.74328938e-24,
    2.61063140e-24,
    2.50640248e-24,
    2.38164559e-24,
    2.26967531e-24,
    2.24509826e-24,
    2.12378192e-24,
    2.02063266e-24,
    1.92509873e-24,
    1.83070678e-24,
    1.77346374e-24,
]

# because pandas reads in the columns as strings, we need to convert them back to integers
bf_heating_estimator.columns = bf_heating_estimator.columns.astype(int)
stim_recomb_cooling_estimator.columns = (
    stim_recomb_cooling_estimator.columns.astype(int)
)
coll_deexc_heating_estimator.columns = coll_deexc_heating_estimator.columns.astype(int)


In [ ]:
continuum_estimators = {}

continuum_estimators['photo_ion_estimator'] = photo_ion_estimator.loc[:, [0]].values
continuum_estimators['stim_recomb_estimator'] = stim_recomb_estimator.loc[:, [0]].values
continuum_estimators['bf_heating_estimator'] = bf_heating_estimator.loc[:, [0]].values
continuum_estimators['stim_recomb_cooling_estimator'] = stim_recomb_cooling_estimator.loc[:, [0]].values
continuum_estimators['coll_deexc_heating_estimator'] = coll_deexc_heating_estimator.loc[:, [0]].values
continuum_estimators['ff_heating_estimator'] = [ff_heating_estimator[0]]

In [ ]:
plasma.update_radiationfield(
            radiation_temp, dilution_factor, j_blues_ctardis["0"],
            config.plasma.nlte, initialize_nlte=False,
            n_e_convergence_threshold=0.05, **continuum_estimators)

In [ ]:
plasma.ion_number_density

In [ ]:
plasma.level_number_density.head()

In [ ]:
plasma.electron_densities